In [ ]:
import os
import shutil
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score
import xgboost as xgb
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

nltk.download('punkt')

In [ ]:
def mover_archivos_a_plagiados(archivos_plagiados, ruta_carpeta_origen, ruta_carpeta_destino):
    if not os.path.exists(ruta_carpeta_destino):  # Crear carpeta en caso de que no exista
        os.makedirs(ruta_carpeta_destino)

    # Mover los archivos a la carpeta de destino
    for archivo in archivos_plagiados:
        ruta_origen = os.path.join(ruta_carpeta_origen, archivo)
        ruta_destino = os.path.join(ruta_carpeta_destino, archivo)
        
        # Checar si el archivo ya se encuentra en el lugar
        if os.path.exists(ruta_destino):
            print(f"El archivo '{archivo}' ya existe en '{ruta_carpeta_destino}', se omitirá.")
            continue
        
        shutil.move(ruta_origen, ruta_destino)
        print(f"Archivo '{archivo}' movido a '{ruta_carpeta_destino}'.")

# Lista de nombres de archivos plagiados proporcionada
archivos_plagiados = [
    "003.java", "004.java", "005.java", "006.java", "008.java", "010.java",
    "014.java", "021.java", "015.java", "023.java", "016.java", "024.java",
    "017.java", "022.java", "030.java", "032.java", "033.java", "034.java",
    "042.java", "044.java", "043.java", "251.java", "045.java", "047.java",
    "048.java", "051.java", "183.java", "257.java", "258.java", "049.java",
    "050.java", "052.java", "053.java", "059.java", "159.java", "250.java",
    "061.java", "216.java", "062.java", "064.java", "069.java", "070.java",
    "078.java", "079.java", "084.java", "085.java", "086.java", "087.java",
    "155.java", "222.java", "242.java", "243.java", "089.java", "090.java",
    "094.java", "098.java", "101.java", "212.java", "103.java", "105.java",
    "106.java", "111.java", "107.java", "108.java", "112.java", "113.java",
    "117.java", "119.java", "131.java", "133.java", "135.java", "174.java",
    "136.java", "173.java", "137.java", "171.java", "140.java", "142.java",
    "143.java", "145.java", "146.java", "147.java", "148.java", "150.java",
    "153.java", "158.java", "161.java", "175.java", "180.java", "181.java",
    "182.java", "185.java", "188.java", "190.java", "191.java", "193.java",
    "195.java", "218.java", "201.java", "209.java", "202.java", "208.java",
    "211.java", "221.java", "224.java", "228.java", "230.java", "232.java",
    "233.java", "235.java", "237.java", "238.java", "240.java", "244.java",
    "246.java"
]

# Ruta de la carpeta compartida que contiene los archivos de código fuente
ruta_carpeta_origen = '.\\archivos\\no_plagiados'

# Ruta de la carpeta donde se moverán los archivos plagiados
ruta_carpeta_destino = '.\\archivos\\plagiados'

# Llamar a la función para mover los archivos
mover_archivos_a_plagiados(archivos_plagiados, ruta_carpeta_origen, ruta_carpeta_destino)

In [ ]:
# Función para leer archivos de código fuente y tokenizarlos
def leer_y_tokenizar_archivo(ruta):
    with open(ruta, 'r') as archivo:
        code = archivo.read()

    # Tokenización utilizando NLTK
    tokens = nltk.word_tokenize(code)

    return tokens, os.path.basename(ruta)  # Devolver también el nombre del archivo

# Ruta de la carpeta de entrenamiento que contiene dos subcarpetas: plagiados y no plagiados
ruta_carpeta_entrenamiento = '.\\archivos'

# Leer archivos de código fuente de la carpeta de entrenamiento y tokenizarlos
datos_entrenamiento = []
for etiqueta, carpeta in enumerate(["no_plagiados", "plagiados"]):
    ruta_carpeta = os.path.join(ruta_carpeta_entrenamiento, carpeta)
    for archivo in os.listdir(ruta_carpeta):
        if archivo.endswith(".java"):
            ruta_archivo = os.path.join(ruta_carpeta, archivo)
            tokens, nombre_archivo = leer_y_tokenizar_archivo(ruta_archivo)
            datos_entrenamiento.append((tokens, etiqueta, nombre_archivo))  # Agrega los tokens, su etiqueta y el nombre del archivo

# Separar datos en características (X), etiquetas (y) y nombres de archivo
X = [(tokens, nombre_archivo) for tokens, _, nombre_archivo in datos_entrenamiento]
y = [etiqueta for _, etiqueta, _ in datos_entrenamiento]

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
#, random_state=42
# Convertir las listas de tokens en cadenas de texto
X_train_text = [' '.join(tokens) for tokens, _ in X_train]
X_val_text = [' '.join(tokens) for tokens, _ in X_val]

# Inicializar el vectorizador TF-IDF
vectorizer = TfidfVectorizer()

# Ajustar el vectorizador y transformar los datos de entrenamiento
X_train_tfidf = vectorizer.fit_transform(X_train_text)

# Transformar los datos de validación utilizando el mismo vectorizador
X_val_tfidf = vectorizer.transform(X_val_text)

# Definir y entrenar el modelo XGBoost
modelo = xgb.XGBClassifier()
modelo.fit(X_train_tfidf, y_train)

# Predecir etiquetas para el conjunto de validación
y_val_pred = modelo.predict(X_val_tfidf)

# Calcular la precisión y la recuperación en el conjunto de validación
precision = precision_score(y_val, y_val_pred)
recuperacion = recall_score(y_val, y_val_pred)

print("Precisión en el conjunto de validación:", precision)
print("Recuperación en el conjunto de validación:", recuperacion)

In [ ]:
# Obtener algunas predicciones para la visualización
num_instancias_visualizacion = 259
instancias_visualizacion = list(zip(X_val_text[:num_instancias_visualizacion], X_val[:num_instancias_visualizacion], y_val[:num_instancias_visualizacion], y_val_pred[:num_instancias_visualizacion]))

# Imprimir las instancias de visualización con etiquetas reales y predichas
print("Instancias de visualización:")
for i, (texto, (_, nombre_archivo), etiqueta_real, etiqueta_predicha) in enumerate(instancias_visualizacion, start=1):
    print(f"Instancia {i}:")
    print("Nombre del Archivo:", nombre_archivo)
    print("Texto:")
    print(texto)
    print("Etiqueta Real:", etiqueta_real)
    print("Etiqueta Predicha:", etiqueta_predicha)
    print("-" * 50)


In [ ]:
# Guardar el modelo entrenado
modelo.save_model("plagio_xgboost.json")

In [16]:
import os
import javalang
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def leer_archivo(archivo):
    with open(archivo, 'r') as f:
        lineas = f.readlines()
        contenido = ''.join([linea for linea in lineas if not linea.strip().startswith("import")]).replace('\n', '')
    return contenido

def limpiar_codigo(codigo):
    codigo = re.sub(r'//.*?\n|/\*.*?\*/', '', codigo, flags=re.DOTALL)
    codigo = re.sub(r'\s+', ' ', codigo)
    codigo = re.sub(r'\n\s+', '\n', codigo)
    return codigo

def construir_ast(codigo):
    try:
        return javalang.parse.parse(codigo)
    except javalang.parser.JavaSyntaxError as e:
        print("Error de sintaxis:", e)

def obtener_archivos_java(ruta_carpeta):
    return [archivo for archivo in os.listdir(ruta_carpeta) if archivo.endswith(".java")]

ruta_carpeta = ".\\CFiles\\java\\train"
archivos_java = obtener_archivos_java(ruta_carpeta)
arboles = {}

for archivo in archivos_java:
    ruta_archivo = os.path.join(ruta_carpeta, archivo)
    contenido_archivo = leer_archivo(ruta_archivo)
    arbol_sintactico = construir_ast(contenido_archivo)
    if arbol_sintactico:
        arboles[archivo] = limpiar_codigo(str(arbol_sintactico))
    else:
        print("No se pudo construir el AST para el archivo", archivo)

vectorizer = TfidfVectorizer()
documentos = [arbol for arbol in arboles.values()]
nombres_archivos = list(arboles.keys())
tfidf_matrix = vectorizer.fit_transform(documentos)
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

umbral_similitud_cosine = 0.98
umbral_similitud_jaccard = 0.5

archivos_similares = []

print("Archivos similares:")
for i, nombre_archivo1 in enumerate(nombres_archivos):
    for j, nombre_archivo2 in enumerate(nombres_archivos):
        if i < j and similarity_matrix[i, j] >= umbral_similitud_cosine:
            tokens1 = set(re.findall(r'\b\w+\b', arboles[nombre_archivo1]))
            tokens2 = set(re.findall(r'\b\w+\b', arboles[nombre_archivo2]))
            similitud_jaccard = len(tokens1.intersection(tokens2)) / len(tokens1.union(tokens2))
            if similitud_jaccard >= umbral_similitud_jaccard:
                archivos_similares.append((nombre_archivo1, nombre_archivo2))
                print(f"- {nombre_archivo1} y {nombre_archivo2} (Similitud de coseno: {similarity_matrix[i, j]:.2f}, Similitud de Jaccard: {similitud_jaccard:.2f})")


Error de sintaxis: 
No se pudo construir el AST para el archivo 00af3420.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 00c0b82a.java


Error de sintaxis: 
No se pudo construir el AST para el archivo 0b04b41e.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 0c1143f7.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 0c9d4def.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 1ea771ea.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 2a35cd81.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 2a655afe.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 3e6def38.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 4fb09c5f.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 548ffb07.java
Error de sintaxis: 
No se pudo construir el AST para el archivo 692a4496.java
Error de sintaxis: 
No se pudo construir el AST para el archivo c4ca2ff3.java
Archivos similares:
- 11c2ab99.java y bdfe8110.java (Similitud de coseno: 1.00, Similitud de Jaccard: 0.98)
- 4da08761.java y 5449d33c.java (S

In [14]:
archivos_plagiados = [
    "003.java", "004.java", "005.java", "006.java", "008.java", "010.java",
    "014.java", "021.java", "015.java", "023.java", "016.java", "024.java",
    "017.java", "022.java", "030.java", "032.java", "033.java", "034.java",
    "042.java", "044.java", "043.java", "251.java", "045.java", "047.java",
    "048.java", "051.java", "183.java", "257.java", "258.java", "049.java",
    "050.java", "052.java", "053.java", "059.java", "159.java", "250.java",
    "061.java", "216.java", "062.java", "064.java", "069.java", "070.java",
    "078.java", "079.java", "084.java", "085.java", "086.java", "087.java",
    "155.java", "222.java", "242.java", "243.java", "089.java", "090.java",
    "094.java", "098.java", "101.java", "212.java", "103.java", "105.java",
    "106.java", "111.java", "107.java", "108.java", "112.java", "113.java",
    "117.java", "119.java", "131.java", "133.java", "135.java", "174.java",
    "136.java", "173.java", "137.java", "171.java", "140.java", "142.java",
    "143.java", "145.java", "146.java", "147.java", "148.java", "150.java",
    "153.java", "158.java", "161.java", "175.java", "180.java", "181.java",
    "182.java", "185.java", "188.java", "190.java", "191.java", "193.java",
    "195.java", "218.java", "201.java", "209.java", "202.java", "208.java",
    "211.java", "221.java", "224.java", "228.java", "230.java", "232.java",
    "233.java", "235.java", "237.java", "238.java", "240.java", "244.java",
    "246.java"
]

# Contar los diferentes tipos de resultados
falsos_positivos = len([par for par in archivos_similares if par[0] not in archivos_plagiados and par[1] in archivos_plagiados])
falsos_negativos = len([archivo in archivos_plagiados for archivo in archivos_plagiados])
verdaderos_positivos = len(archivos_similares)

print("Resultados:")
print(f"Falsos positivos: {falsos_positivos}")
print(f"Falsos negativos: {falsos_negativos}")
print(f"Aciertos: {verdaderos_positivos}")


Resultados:
Falsos positivos: 2
Falsos negativos: 115
Aciertos: 41
